<a href="https://colab.research.google.com/github/Lucy-wo/API-Challenge/blob/master/Final_project_Lucy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMA831 Final Project: Instacart Market Basket Analysis

## Goal: 
 - This is a classification problem because we need to predict whether each pair of user and product is a reorder or not
 - To predict which products will be in a user's next order. i.e. reordered=1 or reordered=0
 - We will do so by analysing the prior orders of the dataset. We will then use the train users to create a predictive model and the test users to make our actual prediction
 - PCA to reduction variables and clustering customer

## Datasets:

Instacart is an American company that operates as a same-day grocery delivery service. Customers select groceries through a web application from various retailers and delivered by a personal shopper. Instacart's service is mainly provided through a smartphone app, available on iOS and Android platforms, apart from its website.

To predict which previously purchased products (prior orders) will be in a user’s next order (train and test orders).

For the train orders Instacart reveals the results (i.e., the ordered products) while for the test orders we do not have this piece of information.

1. orders: All orders, namely prior, train, and test. It has single primary key (order_id).

2. order_products_train: This table includes training orders. It has a composite primary key (order_id and product_id) and indicates whether a product in an order is a reorder or not (through the reordered variable).

3. order_products_prior : This table includes prior orders. It has a composite primary key (order_id and product_id) and indicates whether a product in an order is a reorder or not (through the reordered variable).

4. products: This table includes all products. It has a single primary key (product_id)

5. aisles: This table includes all aisles. It has a single primary key (aisle_id)

6. departments: This table includes all departments. It has a single primary key (department_id)

## Dataset relationship
- users are identified by user_id in the orders csv file. Each row of the orders csv fil represents an order made by a user. Order are identified by order_id;

- Each order of a user is characterized by an order_number which specifies when it has been made with respect to the others of the same user;

- each order consists of a set of product each characterized by an add_to_cart_order feature representing the sequence in which they have been added to the cart in that order;

- for each user we may have n-1 prior orders and 1 train order OR n-1 prior orders and 1 test order in which we have to state what products have been reordered.

# Steps
## One: Build Model 
1. Merge all tables together based on primary keys
2. Data cleasing(missing, colinarity, correlated.....)
3. Feature engineering,Transactional data (such as credit card purchases) 
  - needs to be aggregated in some way to give a concise summary for each user (e.g., average transactions per month, average spend per month, highest spend, most popular store, etc.)
  - Scale data
4. Split train and test datasets
5. Models (KNN need to scale) and apply prediction

## Two: PCA and segment customer


# <font color='blue'>Load Data</font> 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
aisles_df = pd.read_csv("aisles.csv")
departments_df = pd.read_csv("departments.csv")
order_products_prior_df = pd.read_csv("order_products__prior.csv")
order_products_train_df = pd.read_csv("order_products__train.csv")
orders_df = pd.read_csv("orders.csv")
products_df = pd.read_csv("products.csv")

In [4]:
aisles_df['aisle'] = aisles_df['aisle'].astype('category')
departments_df['department'] = departments_df['department'].astype('category')
orders_df['eval_set'] = orders_df['eval_set'].astype('category')
products_df['product_name'] = products_df['product_name'].astype('category')

# <font color='blue'>Check tables information</font> 

In [5]:
products_df.info()
products_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   product_id     49688 non-null  int64   
 1   product_name   49688 non-null  category
 2   aisle_id       49688 non-null  int64   
 3   department_id  49688 non-null  int64   
dtypes: category(1), int64(3)
memory usage: 3.0 MB


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
len(products_df)

49688

In [7]:
aisles_df.info()
aisles_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   aisle_id  134 non-null    int64   
 1   aisle     134 non-null    category
dtypes: category(1), int64(1)
memory usage: 7.5 KB


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [8]:
departments_df.info()
departments_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   department_id  21 non-null     int64   
 1   department     21 non-null     category
dtypes: category(1), int64(1)
memory usage: 1.1 KB


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [9]:
order_products_train_df.info()
order_products_train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602693 entries, 0 to 602692
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   order_id           602693 non-null  int64  
 1   product_id         602692 non-null  float64
 2   add_to_cart_order  602692 non-null  float64
 3   reordered          602692 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 18.4 MB


,order_id,product_id,add_to_cart_order,reordered
0,1,49302.0,1.0,1.0
1,1,11109.0,2.0,1.0
2,1,10246.0,3.0,0.0
3,1,49683.0,4.0,0.0
4,1,43633.0,5.0,1.0


In [10]:
order_products_prior_df.info()
order_products_prior_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591557 entries, 0 to 591556
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   order_id           591557 non-null  int64  
 1   product_id         591556 non-null  float64
 2   add_to_cart_order  591556 non-null  float64
 3   reordered          591556 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 18.1 MB


,order_id,product_id,add_to_cart_order,reordered
0,2,33120.0,1.0,1.0
1,2,28985.0,2.0,1.0
2,2,9327.0,3.0,0.0
3,2,45918.0,4.0,1.0
4,2,30035.0,5.0,0.0


In [11]:
orders_df.info()
orders_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339872 entries, 0 to 339871
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   order_id                339872 non-null  int64   
 1   user_id                 339872 non-null  int64   
 2   eval_set                339872 non-null  category
 3   order_number            339871 non-null  float64 
 4   order_dow               339871 non-null  float64 
 5   order_hour_of_day       339871 non-null  float64 
 6   days_since_prior_order  319289 non-null  float64 
dtypes: category(1), float64(4), int64(2)
memory usage: 15.9 MB


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1.0,2.0,8.0,NaN
1,2398795,1,prior,2.0,3.0,7.0,15.0
2,473747,1,prior,3.0,3.0,12.0,21.0
3,2254736,1,prior,4.0,4.0,7.0,29.0
4,431534,1,prior,5.0,4.0,15.0,28.0


In [12]:
orders_df.groupby('eval_set').nunique()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
eval_set,,,,,,
prior,319290,20582,99,7,24,31
te,1,1,0,0,0,0
test,7418,7418,94,7,24,31
train,13163,13163,96,7,24,31


In [13]:
orders_df.user_id.nunique()

20582

# <font color='blue'>Merge all prior info into one table </font> 

In [14]:
order_products_prior_df = pd.merge(order_products_prior_df, products_df, on='product_id', how='left')

order_products_prior_df = pd.merge(order_products_prior_df, aisles_df, on='aisle_id', how='left')

order_products_prior_df = pd.merge(order_products_prior_df, departments_df, on='department_id', how='left')

order_products_prior_df = pd.merge(order_products_prior_df, orders_df, on='order_id', how='left')

order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120.0,1.0,1.0,Organic Egg Whites,86.0,16.0,eggs,dairy eggs,NaN,NaN,NaN,NaN,NaN,NaN
1,2,28985.0,2.0,1.0,Michigan Organic Kale,83.0,4.0,fresh vegetables,produce,NaN,NaN,NaN,NaN,NaN,NaN
2,2,9327.0,3.0,0.0,Garlic Powder,104.0,13.0,spices seasonings,pantry,NaN,NaN,NaN,NaN,NaN,NaN
3,2,45918.0,4.0,1.0,Coconut Butter,19.0,13.0,oils vinegars,pantry,NaN,NaN,NaN,NaN,NaN,NaN
4,2,30035.0,5.0,0.0,Natural Sweetener,17.0,13.0,baking ingredients,pantry,NaN,NaN,NaN,NaN,NaN,NaN


# <font color='blue'>Feature Engineering</font> 

1. Get each user's total number of orders

In [15]:
user = order_products_prior_df.groupby('user_id')['order_number'].max().to_frame('user_total_orders')
user = user.reset_index()
user.head()

,user_id,user_total_orders
0,7.0,17.0
1,13.0,4.0
2,21.0,5.0
3,23.0,2.0
4,27.0,63.0


2. Get the frequent a customer has reordered products

In [16]:
u_reorder = order_products_prior_df.groupby('user_id')['reordered'].mean().to_frame('user_reordered_ratio')
u_reorder = u_reorder.reset_index()
u_reorder.head()

,user_id,user_reordered_ratio
0,7.0,0.818182
1,13.0,0.600000
2,21.0,0.428571
3,23.0,0.000000
4,27.0,0.461538


In [17]:
user_info = user.merge(u_reorder, on='user_id', how='left')
del u_reorder
user_info.head()

,user_id,user_total_orders,user_reordered_ratio
0,7.0,17.0,0.818182
1,13.0,4.0,0.600000
2,21.0,5.0,0.428571
3,23.0,2.0,0.000000
4,27.0,63.0,0.461538


3. get the number of purchases for each product

In [18]:
prd = order_products_prior_df.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
prd = prd.reset_index()
prd.head()

,product_id,p_total_purchases
0,1.0,42
1,2.0,2
2,3.0,3
3,4.0,5
4,9.0,1


4. The probability for a product to be reordered(Remove products with less than 40 purchases)

In [19]:
p_reorder = order_products_prior_df.groupby('product_id').filter(lambda x: x.shape[0] >40)

p_reorder = p_reorder.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
p_reorder = p_reorder.reset_index()
p_reorder.head()

,product_id,p_reorder_ratio
0,1.0,0.690476
1,34.0,0.540323
2,45.0,0.662011
3,79.0,0.560976
4,95.0,0.196429


In [20]:
prd = prd.merge(p_reorder, on='product_id', how='left')
#deal with NA
prd['p_reorder_ratio'] = prd['p_reorder_ratio'].fillna(value=0)
prd.head()

,product_id,p_total_purchases,p_reorder_ratio
0,1.0,42,0.690476
1,2.0,2,0.000000
2,3.0,3,0.000000
3,4.0,5,0.000000
4,9.0,1,0.000000


5. How many times a user bought a product

In [21]:
uxp = order_products_prior_df.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_total_bought')
uxp = uxp.reset_index()
uxp

,user_id,product_id,uxp_total_bought
0,7.0,4920.0,1
1,7.0,4945.0,1
2,7.0,8277.0,1
3,7.0,11520.0,1
4,7.0,13198.0,1
...,...,...,...
55523,20576.0,24532.0,1
55524,20576.0,26604.0,1
55525,20576.0,27966.0,1
55526,20576.0,39275.0,1


6. frequently a customer bought a product after its first purchase

In [22]:
#each user each product order time
times = order_products_prior_df.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
times

Times_Bought_N
user_id product_id                
7.0     4920.0                   1
        4945.0                   1
        8277.0                   1
        11520.0                  1
        13198.0                  1
...                            ...
20576.0 24532.0                  1
        26604.0                  1
        27966.0                  1
        39275.0                  1
        48812.0                  1

[55528 rows x 1 columns]

In [23]:
#Each user's total order time
total_orders = order_products_prior_df.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders.head()

,total_orders
user_id,
7.0,17.0
13.0,4.0
21.0,5.0
23.0,2.0
27.0,63.0


In [24]:
first_order_no = order_products_prior_df.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()

,user_id,product_id,first_order_number
0,7.0,4920.0,17.0
1,7.0,4945.0,17.0
2,7.0,8277.0,17.0
3,7.0,11520.0,17.0
4,7.0,13198.0,17.0


In [25]:
order_number_info = pd.merge(total_orders, first_order_no, on='user_id', how='right')
order_number_info['Order_Range_D'] = order_number_info.total_orders - order_number_info.first_order_number + 1 # The +1 includes in the difference the first order were the product has been purchased

uxp_ratio = pd.merge(times, order_number_info, on=['user_id', 'product_id'], how='left')
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.Times_Bought_N / uxp_ratio.Order_Range_D
uxp_ratio.head()

,user_id,product_id,Times_Bought_N,total_orders,first_order_number,Order_Range_D,uxp_reorder_ratio
0,7.0,4920.0,1,17.0,17.0,1.0,1.0
1,7.0,4945.0,1,17.0,17.0,1.0,1.0
2,7.0,8277.0,1,17.0,17.0,1.0,1.0
3,7.0,11520.0,1,17.0,17.0,1.0,1.0
4,7.0,13198.0,1,17.0,17.0,1.0,1.0


For example: for user1, product 13023:
total bought it 3 times
user 1 total bought from instacart 10 times
the 2nd time bouth product 13021
so the range belttwen frist time order to last time oder is 9

In [26]:
uxp_ratio = uxp_ratio.drop(['Times_Bought_N', 'total_orders', 'first_order_number', 'Order_Range_D'], axis=1)
uxp_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,7.0,4920.0,1.0
1,7.0,4945.0,1.0
2,7.0,8277.0,1.0
3,7.0,11520.0,1.0
4,7.0,13198.0,1.0


# <font color='blue'>Merge new features with DF</font> 

In [27]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')
del uxp_ratio
uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio
0,7.0,4920.0,1,1.0
1,7.0,4945.0,1,1.0
2,7.0,8277.0,1,1.0
3,7.0,11520.0,1,1.0
4,7.0,13198.0,1,1.0


In [28]:
data = uxp.merge(user, on='user_id', how='left')
data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,user_total_orders,p_total_purchases,p_reorder_ratio
0,7.0,4920.0,1,1.0,17.0,1494,0.670013
1,7.0,4945.0,1,1.0,17.0,99,0.727273
2,7.0,8277.0,1,1.0,17.0,1600,0.751875
3,7.0,11520.0,1,1.0,17.0,731,0.764706
4,7.0,13198.0,1,1.0,17.0,239,0.648536


# <font color='blue'>EDA</font> 

## Plot the count of the Max Order Number for each User

In [ ]:
#find the max order number for each user_id
max_order_number = orders_df.groupby("user_id")["order_number"].aggregate(np.max).reset_index()
max_order_number

,user_id,order_number
0,1,11
1,2,15
2,3,13
3,4,6
4,5,5
...,...,...
206204,206205,4
206205,206206,68
206206,206207,17
206207,206208,50


In [ ]:
max_order_number = max_order_number.order_number.value_counts()
max_order_number

4     23986
5     19590
6     16165
7     13850
8     11700
      ...  
94       57
91       56
97       54
98       50
99       47
Name: order_number, Length: 97, dtype: int64

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.3, color = 'green')
plt.ylabel('Counts', fontsize=12)
plt.xlabel('Maximum order number', fontsize=12)
plt.xticks(rotation='vertical')
plt.title('The distrubution: the occurance of the max order number')
plt.show()

NameError: name 'cnt_srs' is not defined

<Figure size 864x576 with 0 Axes>

## Plot Heatmap of the count of order on datetime during the week

In [ ]:
grouped_df = orders_df.groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
grouped_df

In [ ]:
grouped_df = grouped_df.pivot('order_dow', 'order_hour_of_day', 'order_number')
grouped_df

In [ ]:
#bases on the heatmap, the week start with Sat
plt.figure(figsize=(12,6))
sns.heatmap(grouped_df, yticklabels=['Sat','Sun', 'Mon','Tue','Wed','Thu','Fri'])
plt.title("Heatmap of the count of order on datetime during the week")
plt.ylabel("Time of the day")
plt.xlabel("Day of the week")
plt.show()

In [ ]:
order_products_prior_df['order_dow'].value_counts().sort_values(ascending = False).plot(kind='bar',color='darkcyan')

## Plot the frequency distribution by days since prior order

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(y="days_since_prior_order", data=orders_df, palette="Set1")
plt.ylabel('Days since prior order', fontsize=12)
plt.xlabel('Frequency', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency distribution by days since prior order", fontsize=15)
plt.show()

## Plot the count of product name

In [ ]:
from wordcloud import WordCloud,ImageColorGenerator

import matplotlib.pyplot as plt

#making of word cloud from Product name column
text = " ".join(topic for topic in order_products_prior_df.product_name.astype(str))
print ("There are {} words in the combination of all Product Name.".format(len(text)))

# Create and generate a word cloud image:
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(text)

plt.axis("off")
plt.rcParams["figure.figsize"] = (15,6)
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))

sns.countplot(x='reordered',data=order_products_prior_df,color="c")

In [ ]:
from wordcloud import WordCloud,ImageColorGenerator

import matplotlib.pyplot as plt

#making of word cloud from Product name column
text = " ".join(topic for topic in aisles_df.aisle.astype(str))
print ("There are {} words in the combination of all Product Name.".format(len(text)))

# Create and generate a word cloud image:
wordcloud = WordCloud(background_color="white", width=800, height=400).generate(text)

plt.axis("off")
plt.rcParams["figure.figsize"] = (15,6)
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

In [ ]:
order_products_prior_df.columns

## Plot department counts

In [ ]:
order_products_prior_df['department'].value_counts()

In [ ]:
cnt_srs = order_products_prior_df['department'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Department', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
temp_series = order_products_prior_df['department'].value_counts()
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))
plt.pie(sizes, labels=labels, 
        autopct='%1.1f%%', startangle=200)
plt.title("Departments distribution", fontsize=15)
plt.show()

In [ ]:
order_products_prior_df[order_products_prior_df.department =='dairy eggs'].head()

In [ ]:
order_products_prior_df.head()

In [ ]:
gr_dept_df = order_products_prior_df[order_products_prior_df.department =='dairy eggs'].groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()

## Order ditrubtuion during the week for each Department:

In [ ]:
#Order ditrubtuion during the week for each Department:
for i in order_products_prior_df['department'].value_counts().index:
    gr_dept_df = order_products_prior_df[order_products_prior_df.department==i].groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
    gr_dept_df = gr_dept_df.pivot('order_dow', 'order_hour_of_day', 'order_number')

    plt.figure(figsize=(12,6))
    sns.heatmap(gr_dept_df,yticklabels=['Sat','Sun', 'Mon','Tue','Wed','Thu','Fri'])
    plt.title("Order ditrubtuion during the week for each Department: "+i)
    plt.ylabel("The day of week")
    plt.xlabel("The time of day")
    plt.show()

In [ ]:
gr_dept_df

# <font color='blue'>Dummy Variables </font> 

In [ ]:
dummies_df = pd.get_dummies(data=order_products_prior_df, prefix=['Day','Hour'], columns=['order_dow','order_hour_of_day'], drop_first=True)
dummies_df.head(10)

In [ ]:
dummies_df.columns

In [ ]:
user_prod_df = dummies_df.groupby(['user_id','product_id']).agg({'order_id':'nunique',
                                                                 'days_since_prior_order':'mean',
                                                                 'reordered':'max',
                                                                 'Day_1':'sum',
                                                                 'Day_2':'sum',
                                                                 'Day_3':'sum',
                                                                 'Day_4':'sum',
                                                                 'Day_5':'sum',
                                                                 'Day_6':'sum',
                                                                 'Hour_1':'sum',
                                                                 'Hour_2':'sum',
                                                                 'Hour_3':'sum',
                                                                 'Hour_4':'sum',
                                                                 'Hour_5':'sum',
                                                                 'Hour_6':'sum',
                                                                 'Hour_7':'sum',
                                                                 'Hour_8':'sum',
                                                                 'Hour_9':'sum',
                                                                 'Hour_10':'sum',
                                                                 'Hour_11':'sum',
                                                                 'Hour_12':'sum',
                                                                 'Hour_13':'sum',
                                                                 'Hour_14':'sum',
                                                                 'Hour_15':'sum',
                                                                 'Hour_16':'sum',
                                                                 'Hour_17':'sum',
                                                                 'Hour_18':'sum',
                                                                 'Hour_19':'sum',
                                                                 'Hour_20':'sum',
                                                                 'Hour_21':'sum',
                                                                 'Hour_22':'sum',
                                                                 'Hour_23':'sum'
                                                                }).reset_index()
user_prod_df.head(10)

In [ ]:
user_purchase_df = dummies_df.groupby(['user_id']).agg({         'order_id':'nunique',
                                                                 'product_id': 'nunique',
                                                                 'days_since_prior_order':'mean',
                                                                 'reordered':'sum',
                                                                 'Day_1':'sum',
                                                                 'Day_2':'sum',
                                                                 'Day_3':'sum',
                                                                 'Day_4':'sum',
                                                                 'Day_5':'sum',
                                                                 'Day_6':'sum',
                                                                 'Hour_1':'sum',
                                                                 'Hour_2':'sum',
                                                                 'Hour_3':'sum',
                                                                 'Hour_4':'sum',
                                                                 'Hour_5':'sum',
                                                                 'Hour_6':'sum',
                                                                 'Hour_7':'sum',
                                                                 'Hour_8':'sum',
                                                                 'Hour_9':'sum',
                                                                 'Hour_10':'sum',
                                                                 'Hour_11':'sum',
                                                                 'Hour_12':'sum',
                                                                 'Hour_13':'sum',
                                                                 'Hour_14':'sum',
                                                                 'Hour_15':'sum',
                                                                 'Hour_16':'sum',
                                                                 'Hour_17':'sum',
                                                                 'Hour_18':'sum',
                                                                 'Hour_19':'sum',
                                                                 'Hour_20':'sum',
                                                                 'Hour_21':'sum',
                                                                 'Hour_22':'sum',
                                                                 'Hour_23':'sum'
                                                                }).reset_index()
user_purchase_df.head(10)

In [ ]:
product_purchase_df = dummies_df.groupby(['product_id']).agg({   'order_id':'nunique',
                                                                 'user_id': 'nunique',
                                                                 'days_since_prior_order':'mean',
                                                                 'reordered':'sum',
                                                                 'Day_1':'sum',
                                                                 'Day_2':'sum',
                                                                 'Day_3':'sum',
                                                                 'Day_4':'sum',
                                                                 'Day_5':'sum',
                                                                 'Day_6':'sum',
                                                                 'Hour_1':'sum',
                                                                 'Hour_2':'sum',
                                                                 'Hour_3':'sum',
                                                                 'Hour_4':'sum',
                                                                 'Hour_5':'sum',
                                                                 'Hour_6':'sum',
                                                                 'Hour_7':'sum',
                                                                 'Hour_8':'sum',
                                                                 'Hour_9':'sum',
                                                                 'Hour_10':'sum',
                                                                 'Hour_11':'sum',
                                                                 'Hour_12':'sum',
                                                                 'Hour_13':'sum',
                                                                 'Hour_14':'sum',
                                                                 'Hour_15':'sum',
                                                                 'Hour_16':'sum',
                                                                 'Hour_17':'sum',
                                                                 'Hour_18':'sum',
                                                                 'Hour_19':'sum',
                                                                 'Hour_20':'sum',
                                                                 'Hour_21':'sum',
                                                                 'Hour_22':'sum',
                                                                 'Hour_23':'sum'
                                                                }).reset_index()
product_purchase_df.head(10)

In [ ]:
temp = pd.merge(left=user_prod_df,  right=user_purchase_df, on='user_id', suffixes=('','_user'))
temp.head(10)

In [ ]:
features_df = pd.merge(left=temp,  right=product_purchase_df, on='product_id', suffixes=('','_prod'))
features_df.head(10)

In [ ]:
features_df.shape

In [ ]:
features_df.info()

In [ ]:
features_df.isnull().sum()

In [ ]:
features_df.drop(columns=['days_since_prior_order'],inplace= True)

In [ ]:
reduced_feature= features_df[:1000]

In [ ]:
reduced_feature.head()

# <font color='blue'>Split Train and Test </font> 

In [29]:
orders_future = orders_df[((orders_df.eval_set=='train') | (orders_df.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797
53,6,test,1528013
74,7,train,525192
78,8,train,880375
82,9,train,1094988
88,10,train,1822501


In [30]:
df = data.merge(orders_future, on='user_id', how='left')
df.head(10)

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,user_total_orders,p_total_purchases,p_reorder_ratio,eval_set,order_id
0,7.0,4920.0,1,1.0,17.0,1494,0.670013,train,525192
1,7.0,4945.0,1,1.0,17.0,99,0.727273,train,525192
2,7.0,8277.0,1,1.0,17.0,1600,0.751875,train,525192
3,7.0,11520.0,1,1.0,17.0,731,0.764706,train,525192
4,7.0,13198.0,1,1.0,17.0,239,0.648536,train,525192
5,7.0,17638.0,1,1.0,17.0,47,0.723404,train,525192
6,7.0,27344.0,1,1.0,17.0,750,0.665333,train,525192
7,7.0,32177.0,1,1.0,17.0,188,0.457447,train,525192
8,7.0,37602.0,1,1.0,17.0,16,0.000000,train,525192
9,7.0,40852.0,1,1.0,17.0,84,0.785714,train,525192


In [42]:
data_train = df[df.eval_set=='train']
data_train

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,user_total_orders,p_total_purchases,p_reorder_ratio,eval_set,order_id
0,7.0,4920.0,1,1.0,17.0,1494,0.670013,train,525192
1,7.0,4945.0,1,1.0,17.0,99,0.727273,train,525192
2,7.0,8277.0,1,1.0,17.0,1600,0.751875,train,525192
3,7.0,11520.0,1,1.0,17.0,731,0.764706,train,525192
4,7.0,13198.0,1,1.0,17.0,239,0.648536,train,525192
...,...,...,...,...,...,...,...,...,...
55523,20576.0,24532.0,1,0.5,6.0,27,0.000000,train,1680740
55524,20576.0,26604.0,1,1.0,6.0,1139,0.635645,train,1680740
55525,20576.0,27966.0,1,1.0,6.0,2545,0.754028,train,1680740
55526,20576.0,39275.0,1,1.0,6.0,1822,0.621295,train,1680740


In [43]:
data_train = data_train.set_index(['user_id', 'product_id'])
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)
data_train

uxp_total_bought  ...  p_reorder_ratio
user_id product_id                    ...                 
7.0     4920.0                     1  ...         0.670013
        4945.0                     1  ...         0.727273
        8277.0                     1  ...         0.751875
        11520.0                    1  ...         0.764706
        13198.0                    1  ...         0.648536
...                              ...  ...              ...
20576.0 24532.0                    1  ...         0.000000
        26604.0                    1  ...         0.635645
        27966.0                    1  ...         0.754028
        39275.0                    1  ...         0.621295
        48812.0                    1  ...         0.464789

[36527 rows x 5 columns]

Test Dataset

In [33]:
data_test = df[df.eval_set=='test']
data_test = data_test.set_index(['user_id', 'product_id'])
data_test = data_test.drop(['eval_set','order_id'], axis=1)
data_test

uxp_total_bought  ...  p_reorder_ratio
user_id product_id                    ...                 
36.0    1654.0                     1  ...         0.000000
        11079.0                    1  ...         0.000000
        30415.0                    1  ...         0.000000
54.0    6638.0                     1  ...         0.547619
        11913.0                    1  ...         0.500000
...                              ...  ...              ...
20564.0 18465.0                    1  ...         0.719562
        29527.0                    1  ...         0.000000
        35951.0                    1  ...         0.775270
        47672.0                    1  ...         0.589744
        48171.0                    1  ...         0.491071

[19001 rows x 5 columns]

# <font color='blue'>XGBoost</font> 

In [34]:
pip install xgboost

In [40]:
data_train

uxp_total_bought  ...  p_reorder_ratio
user_id product_id                    ...                 
7.0     4920.0                     1  ...         0.670013
        4945.0                     1  ...         0.727273
        8277.0                     1  ...         0.751875
        11520.0                    1  ...         0.764706
        13198.0                    1  ...         0.648536
...                              ...  ...              ...
20576.0 24532.0                    1  ...         0.000000
        26604.0                    1  ...         0.635645
        27966.0                    1  ...         0.754028
        39275.0                    1  ...         0.621295
        48812.0                    1  ...         0.464789

[36527 rows x 5 columns]

In [35]:
import xgboost as xgb

In [39]:
X_train, y_train = data_train, data_train.reordered
parameters = {'eval_metric':'logloss', 
              'max_depth':'5', 
              'colsample_bytree':'0.4',
              'subsample':'0.75'
             }

xgbc = xgb.XGBClassifier(objective='binary:logistic', parameters=parameters, num_boost_round=10)
model = xgbc.fit(X_train, y_train)

ValueError: ignored

In [ ]:
xgb.plot_importance(model)

In [ ]:
model.get_xgb_params()

In [ ]:
# Fine-tune your model

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
  
paramGrid = {"max_depth":[5,10]
            "colsample_bytree":[0.3, 0.4]}  
xgbc = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', num_boost_round=10)
gridsearch = GridSearchCV(xgbc, paramGrid, cv=3, verbose=2, n_jobs=-1)


model = gridsearch.fit(X_train, y_train)
print("The best parameters are: /n",  gridsearch.best_params_)

model = gridsearch.best_estimator_
del [X_train, y_train]

In [ ]:
model.get_params()

In [ ]:
test_pred = model.predict(data_test).astype(int)

## OR Set custom threshold 
test_pred = (model.predict_proba(data_test)[:,1] >= 0.21).astype(int)

test_pred[0:10] #display the first 10 predictions of the numpy array

In [ ]:
data_test['prediction'] = test_pred
data_test.head(10)

In [ ]:
final = data_test.reset_index()
# Keep only the required columns to create our submission file (for chapter 6)
final = final[['product_id', 'user_id', 'prediction']]

gc.collect()
final.head()

# <font color='blue'>Customer segmentation with PCA</font> 

#### Rely on aisles, which represent categories of products. 
#### Even with aisles features will be too much so will use PCA to find new dimensions along which clustering will be easier

In [ ]:
aisles_df = pd.read_csv("aisles.csv")
departments_df = pd.read_csv("departments.csv")
order_products_prior_df = pd.read_csv("order_products__prior.csv")
order_products_train_df = pd.read_csv("order_products__train.csv")
orders_df = pd.read_csv("orders.csv")
products_df = pd.read_csv("products.csv")

In [ ]:
prior = order_products_prior_df
prior.head()

In [ ]:
train = order_products_train_df
train.head()

In [ ]:
prior = prior[0:300000]

In [ ]:
#Merge order data and prior data
order_prior = pd.merge(prior,orders_df,on=['order_id','order_id'])
order_prior = order_prior.sort_values(by=['user_id','order_id'])
order_prior.head()

In [ ]:
products = products_df
products.head()

In [ ]:
aisles = aisles_df
aisles.head()

In [ ]:
print(aisles.shape)

In [ ]:
#Merge prior, order, products and aisles data
_mt = pd.merge(prior,products, on = ['product_id','product_id'])
_mt = pd.merge(_mt,orders_df,on=['order_id','order_id'])
mt = pd.merge(_mt,aisles,on=['aisle_id','aisle_id'])
mt.head(10)

In [ ]:
#Top 20 sells of the product
mt['product_name'].value_counts()[0:20]

In [ ]:
#Total 24836 different kinds of productname
len(mt['product_name'].unique())

In [ ]:
prior.shape

## Clustering Customers
find a possible clusters among the different customers and substitute single user_id with the cluster to which they are assumed to belong.

In [ ]:
#Total 134 kinds of aisle
len(mt['aisle'].unique())

In [ ]:
#Top 20 sells aisle
mt['aisle'].value_counts()[0:20]

In [ ]:
#A table include all the user and their purchased aisles in prior
cust_prod = pd.crosstab(mt['user_id'], mt['aisle'])
cust_prod

In [ ]:
cust_prod.shape

## Principal Component Analysis
To reduce the number of features from the number of aisles to 6, the numbr of principal components I have chosen.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
pca.fit(cust_prod)
pca_samples = pca.transform(cust_prod)

In [ ]:
ps = pd.DataFrame(pca_samples)

In [ ]:
#6 PCA demension for all rows
ps

In [ ]:
#PCS cpmponents from 0 to 5
pca.components_

In [ ]:
len(pca.components_[0])

In [ ]:
len(pca.components_[4])

In [ ]:
d = {
'cat':list(cust_prod.columns),
'pc0': pca.components_[0],
'pc1': pca.components_[1],
'pc2': pca.components_[2],
'pc3': pca.components_[3],
'pc4': pca.components_[4],
'pc5': pca.components_[5],
    
}
pcdf = pd.DataFrame(d)

In [ ]:
#PCA1 by each aisle
pcdf

In [ ]:
pcdf.sort_values(by='pc0')

In [ ]:
display(pcdf.sort_values(by='pc1')[['cat', 'pc1']])

In [ ]:
display(pcdf.sort_values(by='pc2')[['cat', 'pc2']])

In [ ]:
cust_prod.shape

In [ ]:
cust_prod

In [ ]:
ps

In [ ]:
#ps and cust_prod all total 25831 rows
ps.shape

In [ ]:
tocluster = pd.DataFrame(ps)
tocluster

In [ ]:
#use PC4 and PC5
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
tocluster = pd.DataFrame(ps[[4,5]])
print ('tocluster shape is',tocluster.shape)
print (tocluster.head())

fig = plt.figure(figsize=(12,6))
plt.plot(tocluster[4], tocluster[5], '*', markersize=2, color='yellow', alpha=0.5, label='PC4 and PC5')

plt.xlabel('x_values_4')
plt.ylabel('y_values_5')
plt.legend()
plt.show()

In [ ]:
#use PC2 and PC3
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
tocluster = pd.DataFrame(ps[[2,3]])
print ('tocluster shape is',tocluster.shape)
print (tocluster.head())

fig = plt.figure(figsize=(12,6))
plt.plot(tocluster[2], tocluster[3], '*', markersize=2, color='purple', alpha=0.5, label='PC2 and PC3')

plt.xlabel('x_values_2')
plt.ylabel('y_values_3')
plt.legend()
plt.show()

In [ ]:
#use PC4 and PC1
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
tocluster = pd.DataFrame(ps[[4,1]])
print ('tocluster shape is',tocluster.shape)
print (tocluster.head())

fig = plt.figure(figsize=(12,6))
plt.plot(tocluster[4], tocluster[1], '*', markersize=2, color='green', alpha=0.5, label='PC4 and PC1')

plt.xlabel('x_values_4')
plt.ylabel('y_values_1')
plt.legend()
plt.show()

In [ ]:
#combine all together
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
tocluster = pd.DataFrame(ps)
print ('tocluster shape is',tocluster.shape)
print(tocluster)

fig = plt.figure(figsize=(12,6))
plt.plot(tocluster[4], tocluster[5], '*', markersize=2, color='yellow', alpha=0.5, label='PC4 and PC5')
plt.plot(tocluster[2], tocluster[3], '*', markersize=2, color='pink', alpha=0.5, label='PC2 and PC3')
plt.plot(tocluster[4], tocluster[1], '*', markersize=2, color='green', alpha=0.5, label='PC4 and PC1')

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

In [ ]:
tocluster

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusterer = KMeans(n_clusters=4,random_state=42).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)
print(centers)

In [ ]:
c_preds.shape

In [ ]:
print (c_preds[0:200])

## clusters appear to all PCs

In [ ]:
import matplotlib
fig = plt.figure(figsize=(8,8))
colors = ['orange','blue','purple','green']
colored = [colors[k] for k in c_preds]
print (colored[0:10])
plt.scatter(tocluster[4],tocluster[1],  color = colored)
for ci,c in enumerate(centers):
    plt.plot(c[0], c[1], 'o', markersize=8, color='red', alpha=0.9, label=''+str(ci))

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

## use 2,3 paris try clustering, better result

In [ ]:
tocluster = pd.DataFrame(ps[[2,3]])

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusterer = KMeans(n_clusters=4,random_state=42).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)
print(centers)

In [ ]:
import matplotlib
fig = plt.figure(figsize=(8,8))
colors = ['orange','blue','purple','green']
colored = [colors[k] for k in c_preds]
print (colored[0:10])
plt.scatter(tocluster[2],tocluster[3],  color = colored)
for ci,c in enumerate(centers):
    plt.plot(c[0], c[1], 'o', markersize=8, color='red', alpha=0.9, label=''+str(ci))

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

## use 4,5 paris try clustering, better result

In [ ]:
tocluster = pd.DataFrame(ps[[4,5]])

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusterer = KMeans(n_clusters=4,random_state=42).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)
print(centers)

In [ ]:
import matplotlib
fig = plt.figure(figsize=(8,8))
colors = ['orange','blue','purple','green']
colored = [colors[k] for k in c_preds]
print (colored[0:10])
plt.scatter(tocluster[4],tocluster[5],  color = colored)
for ci,c in enumerate(centers):
    plt.plot(c[0], c[1], 'o', markersize=8, color='red', alpha=0.9, label=''+str(ci))

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

## use 4,1 paris try clustering, better result

In [ ]:
tocluster = pd.DataFrame(ps[[4,1]])

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

clusterer = KMeans(n_clusters=4,random_state=42).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)
print(centers)

In [ ]:
import matplotlib
fig = plt.figure(figsize=(8,8))
colors = ['orange','blue','purple','green']
colored = [colors[k] for k in c_preds]
print (colored[0:10])
plt.scatter(tocluster[4],tocluster[1],  color = colored)
for ci,c in enumerate(centers):
    plt.plot(c[0], c[1], 'o', markersize=8, color='red', alpha=0.9, label=''+str(ci))

plt.xlabel('x_values')
plt.ylabel('y_values')
plt.legend()
plt.show()

## PC 45, could explain more dots
Let's check if we also manage to find some interesting pattern beneath it.

In [ ]:
cust_prod

In [ ]:
c_preds
c_preds.shape

In [ ]:
clust_prod = cust_prod.copy()
clust_prod['cluster'] = c_preds

clust_prod.head(10)

# Description of C0, C1, C2 and C3

In [ ]:
print (clust_prod.shape)

f,arr = plt.subplots(2,2,sharex=True,figsize=(15,15))

c1_count = len(clust_prod[clust_prod['cluster']==0])

c0 = clust_prod[clust_prod['cluster']==0].drop('cluster',axis=1).mean()
arr[0,0].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c0)

c1 = clust_prod[clust_prod['cluster']==1].drop('cluster',axis=1).mean()
arr[0,1].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c1)

c2 = clust_prod[clust_prod['cluster']==2].drop('cluster',axis=1).mean()
arr[1,0].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c2)

c3 = clust_prod[clust_prod['cluster']==3].drop('cluster',axis=1).mean()
arr[1,1].bar(range(len(clust_prod.drop('cluster',axis=1).columns)),c3)

plt.show()

In [ ]:
#Cluster 0 top 20 aisle
c0.sort_values(ascending=False)[0:20]

In [ ]:
#Cluster 1 top 20 aisle
c1.sort_values(ascending=False)[0:20]

In [ ]:
#Cluster 2 top 20 aisle
c2.sort_values(ascending=False)[0:20]

In [ ]:
#Cluster 3 top 20 aisle
c3.sort_values(ascending=False)[0:20]

fresh fruits
fresh vegetables
packaged vegetables fruits
yogurt
packaged cheese
milk
water seltzer sparkling water
chips pretzels
are products which are genereically bought by the majority of the customers.

What we can inspect here is if clusters differ in quantities and proportions, with respect of these goods 
Or a cluster is characterized by some goods not included in this list. For instance we can already see cluster 3 is characterized by 'Baby Food Formula' product which is a significant difference with other clusters.

In [ ]:
from IPython.display import display, HTML

cluster_means = [[c0['fresh fruits'],c0['fresh vegetables'],c0['packaged vegetables fruits'], c0['yogurt'], c0['packaged cheese'], c0['milk'],c0['water seltzer sparkling water'],c0['chips pretzels']],
                 [c1['fresh fruits'],c1['fresh vegetables'],c1['packaged vegetables fruits'], c1['yogurt'], c1['packaged cheese'], c1['milk'],c1['water seltzer sparkling water'],c1['chips pretzels']],
                 [c2['fresh fruits'],c2['fresh vegetables'],c2['packaged vegetables fruits'], c2['yogurt'], c2['packaged cheese'], c2['milk'],c2['water seltzer sparkling water'],c2['chips pretzels']],
                 [c3['fresh fruits'],c3['fresh vegetables'],c3['packaged vegetables fruits'], c3['yogurt'], c3['packaged cheese'], c3['milk'],c3['water seltzer sparkling water'],c3['chips pretzels']]]

cluster_means = pd.DataFrame(cluster_means, columns = ['fresh fruits','fresh vegetables','packaged vegetables fruits','yogurt','packaged cheese','milk','water seltzer sparkling water','chips pretzels'])

cluster_perc = cluster_means.iloc[:, :].apply(lambda x: (x / x.sum())*100,axis=1)

In [ ]:
cluster_perc

## The table above decribe the percentage these goods with respect to the other top 8 in each cluster. 

Cluster 1: Buy more fresh vegetables than the other clusters. As shown by absolute data, Cluster 1 is also the cluster including those customers buying far more goods than any others.

Cluster 2: buy more yogurt than people of the other clusters.

Cluster 3: buy a Lot of 'Baby Food Formula' which not even listed in the top 8 products but mainly characterize this cluster. They buy more milk than the others.

## More info: the 10th to 15th most bought products for each cluster which will not include the generic products (i.e. vegetables, fruits, water, etc.) bought by anyone.

In [ ]:
c0.sort_values(ascending=False)[10:15]

In [ ]:
c1.sort_values(ascending=False)[10:15]

In [ ]:
c2.sort_values(ascending=False)[10:15]

In [ ]:
c3.sort_values(ascending=False)[10:15]

# <font color='blue'>Xgboost </font> 

In [ ]:
#one hot not good for xgboost